In [ ]:
import pandas
import pm4py

In [ ]:
event_log = pandas.read_csv('logs/TJSP.csv', sep=',')
#lawsuit_id,date,movement,order,status,area,subject_matter,control,digital,class,distribution_date,court_department,claim_amount,judge
event_log = pm4py.format_dataframe(event_log, case_id='lawsuit_id', activity_key='movement', timestamp_key='date')
start_activities = pm4py.get_start_activities(event_log)
end_activities = pm4py.get_end_activities(event_log)
print("Start activities: {}\nEnd activities: {}".format(start_activities, end_activities))
# the log is filtered on the top 5 variants
#filtered_log = pm4py.filter_variants_top_k(event_log , 5)

In [ ]:
# a directly - follows graph (DFG) is discovered from the log
dfg, start_activities, end_activities = pm4py.discover_dfg(event_log)


In [ ]:
# a process tree is discovered using the inductive miner
process_tree = pm4py.discover_process_tree_inductive(event_log)

In [ ]:
# the process tree is converted to an accepting Petri net
petri_net , initial_marking, final_marking = pm4py.convert_to_petri_net(process_tree)
process_tree = pm4py.discover_process_tree_inductive(event_log)

In [ ]:
# the accepting Petri net is converted to a BPMN diagram
bpmn_diagram = pm4py.convert_to_bpmn(petri_net, initial_marking, final_marking)

In [ ]:
# the discovered process models are shown on the screen as .svg images
pm4py.view_dfg(dfg, start_activities, end_activities, format='png')
pm4py.view_process_tree(process_tree , format='png')
pm4py.view_petri_net(petri_net, initial_marking, final_marking, format='png')
pm4py.view_bpmn(bpmn_diagram, format='png')

In [ ]:
# we compare the original log versus the discovered model
fitness = pm4py.fitness_token_based_replay(event_log, petri_net, initial_marking, final_marking)
precision = pm4py.precision_token_based_replay(event_log, petri_net, initial_marking, final_marking)
print(fitness) # 0.984
print(precision) # 0.758
# we check if the discovered model is a sound workflow net
is_sound_wfnet = pm4py.check_soundness(petri_net, initial_marking, final_marking)[0]
print(is_sound_wfnet) # True